# Objetivo

- Pasar archivos '.csv' to '.json'

# Load libraries

In [92]:
import pandas as pd
import json
import numpy as np
import spacy

In [93]:
import itertools

In [94]:
# Especificar las rutas de los archivos
ruta_csv =  'congreso_preguntas/out_001132.csv'  # Reemplaza esto con la ruta a tu archivo CSV
# Leer el archivo CSV
df_original = pd.read_csv(ruta_csv,sep=";")

In [95]:
# Especificar las rutas de los archivos
ruta_csv =  'congreso_preguntas/relacion_municipios_02.csv'  # Reemplaza esto con la ruta a tu archivo CSV
# Leer el archivo CSV
df_relacion_municipios = pd.read_csv(ruta_csv,sep=";", encoding='latin-1')

In [96]:
# Especificar las rutas de los archivos
ruta_json =  'congreso_preguntas/diputados_diccionario.json' 
# Leer el archivo JSON
with open(ruta_json, 'r') as archivo:
    diputados_dict = json.load(archivo)

In [97]:
new_diputados_dict = {}
for key, diputa_dict in diputados_dict.items():
    new_diputados_dict[diputa_dict['nombre']] = diputa_dict['grupo_parlamentario']

In [162]:
new_new_diputados_dict = {}
for key,values in new_diputados_dict.items():
    new_new_diputados_dict[key.split(',')[1][1:] +' '+key.split(',')[0]] = values
new_new_diputados_dict    

{'Guillermo Antonio Meijón Couselo': 'G.P. Socialista',
 'Luis Carlos Sahuquillo García': 'G.P. Socialista',
 'Mariana de Gracia Canales Duque': 'G.P. Socialista',
 'Antidio Fagúndez Campo': 'G.P. Socialista',
 'Francisco José Salazar Rodríguez': 'G.P. Socialista',
 'José Antonio Rodríguez Salas': 'G.P. Socialista',
 'Carmen Riolobos Regadera': 'G.P. Popular en el Congreso',
 'Elvira Ramón Utrabo': 'G.P. Socialista',
 'Víctor Valentín Píriz Maya': 'G.P. Popular en el Congreso',
 'Elvira Velasco Morillo': 'G.P. Popular en el Congreso',
 'Joaquín María García Díez': 'G.P. Popular en el Congreso',
 'Alberto Garzón Espinosa': 'G.P. Confederal de Unidas Podemos-En Comú Podem-Galicia en Común',
 'Pablo Echenique Robba': 'G.P. Confederal de Unidas Podemos-En Comú Podem-Galicia en Común',
 'Ferran Bel Accensi': 'G.P. Plural',
 'Herminio Rufino Sancho Íñiguez': 'G.P. Socialista',
 'Vicente Tirado Ochoa': 'G.P. Popular en el Congreso',
 'María José García-Pelayo Jurado': 'G.P. Popular en el Cong

# Configuration

In [98]:
pd.set_option('display.max_colwidth', None)

# Functions

In [99]:
def find_tags_in_text (row,search_strings):
    """
    """
    text = row['Contenido']
    tags_list = []
    for s in search_strings:
        if s in text:
            tags_list.append(s)
    return tags_list

In [100]:
def get_entidades_por_texto_list(nlp, texto_list):
    """
    """
    nlp = spacy.load("es_core_news_sm")

    # Lista de textos
    textos = textos_list
    
    # Procesar la lista de textos utilizando nlp.pipe
    docs = nlp.pipe(textos)
    
    # Extraer entidades y generar la lista de lista de tuplas
    lista_de_listas_de_tuplas = []
    for doc in docs:
        lista_tuplas = [(entidad.text, entidad.label_) for entidad in doc.ents]
        lista_de_listas_de_tuplas.append(lista_tuplas)
    return lista_de_listas_de_tuplas

In [140]:
def get_diputado_grupo_parlamentario_listlist(new_new_diputados_dict, df):
    """
    :return diputado_grupo_parlamentario_listlist:
    """
    diputado_list = list(df['diputados_autores'])
    diputado_grupo_parlamentario_listlist = []
    for i in diputado_list:
        diputado_grupo_parlamentario_list = []
        for j in i:
            try:
                diputado_grupo_parlamentario_list.append(new_new_diputados_dict[j])
            except:
                pass
        diputado_grupo_parlamentario_listlist.append(diputado_grupo_parlamentario_list)
    return diputado_grupo_parlamentario_listlist

# Pipeline

In [141]:
# Especificar las rutas de los archivos
ruta_csv =  'congreso_preguntas/out_001132.csv'  # Reemplaza esto con la ruta a tu archivo CSV
# Leer el archivo CSV
df = pd.read_csv(ruta_csv, sep=";")
# Filter
df = df
#Drop 'NaN' values in rows
df = df.dropna()
#Transform 'strings'
df['Presentada'] = df['Presentada'].str.replace(r'Presentado el ', '', regex=True)
df['Calificada'] = df['Calificada'].str.replace(r'Calificado el ', '', regex=True)
#Add  'diputados_autores'column
bruto_list = list(df['Bruto'])
diputados_autores_list = [[i[101:]] for i in bruto_list]
df['diputados_autores'] = [[j.split(',') for j in i] for i in diputados_autores_list]
df['diputados_autores'] = [list(itertools.chain(*i)) for i in list(df['diputados_autores'])]
#Drop 'Bruto' column
df = df.drop(columns=['Bruto'])
df = df.drop(columns=['Tipo'])

In [142]:
# Cargar el modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")
# Lista de textos
textos_list = list(df['Contenido'])
entity_list_01 = get_entidades_por_texto_list(nlp, textos_list[0:50000])
entity_list_02 = get_entidades_por_texto_list(nlp, textos_list[50000:100000])
entity_list_03 = get_entidades_por_texto_list(nlp, textos_list[100000:])

In [143]:
textos_list

['Discriminación de Asturias en la construcción de líneas ferroviarias en relación con el resto de España.',
 'Motivos del desinterés del Gobierno por la mejora de los servicios ferroviarios de Asturias, especialmente de Gijón.',
 'Previsiones acerca de dotar a los consumidores electrointensivos de un marco jurídico y económico para el suministro eléctrico, así como medidas para frenar la destrucción de empleo en dicho sector.',
 'Medidas previstas para resolver el problema de seguridad que sufre el paso a nivel que une las parroquias de Veriña y Poago (Gijón).',
 'Medidas ante el anuncio de la realización de un ERE de extinción por la empresa Expal Metallurgy en Trubia (Oviedo).',
 'Negociaciones para reformar los sistemas de financiación autonómica y local, así como medidas para facilitar el aplazamiento de las devoluciones que correspondan a los ayuntamientos.',
 'Medidas previstas ante el constante incremento del número de jabalíes silvestres en España, especialmente en Asturias.',

In [144]:
#Get 'entity_list'
entity_list = entity_list_01+entity_list_02+entity_list_03

In [145]:
loc_data_list = [[entity for entity in sublist if entity[1] == 'LOC'] for sublist in entity_list]
per_data_list = [[entity for entity in sublist if entity[1] == 'PER'] for sublist in entity_list]
org_data_list = [[entity for entity in sublist if entity[1] == 'ORG'] for sublist in entity_list]
# misc_data_list = [[entity for entity in sublist if entity[1] == 'MISC'] for sublist in entity_list]

In [146]:
# df['loc_tags'] = [[i[0] for i in loc_data_sublist] for loc_data_sublist in loc_data_list]
# df['per_tags'] = [[i[0] for i in per_data_sublist] for per_data_sublist in per_data_list]
# df['org_tags'] = [[i[0] for i in org_data_sublist] for org_data_sublist in org_data_list]

In [147]:
comunidades_tags = ['Andalucía','Aragón','Asturias',\
                    'Balears','Baleares','Canarias',\
                    'Cantabria','Castilla y León','Castilla-La Mancha',\
                    'Cataluña','Comunidad Valenciana','Comunitat Valenciana',\
                    'Extremadura','Galicia','Madrid',\
                    'Murcia','Navarra','País Vasco','Euskadi',\
                    'Ceuta','Melilla','La Rioja']  # Lista de strings a buscar
# Añadir la nueva columna al DataFrame utilizando lambda para pasar search_strings
df['comunidades_tags'] = df.apply(lambda row: find_tags_in_text(row, comunidades_tags), axis=1)

In [148]:
# search_strings = ['Asturias','Madrid','Málaga']  # Lista de strings a buscar
# df['ccaa_tags'] = df['Contenido'].apply(find_string_in_text)
provincia_tags = ['Albacete','Alicante','Alacant','Almería','Araba','Álava',\
                'Asturias','Ávila','Badajoz','Balears, Illes','Barcelona',\
                'Bizkaia','Burgos','Cáceres','Cádiz','Cantabria','Castellón',\
                'Castelló','Ciudad Real','Córdoba','Coruña','Cuenca','Eivissa','Gipuzkoa',\
                'Girona','Granada','Guadalajara','Huelva','Huesca','Jaén','León',\
                'Lleida','Lugo','Madrid','Málaga','Mallorca','Menorca','Murcia','Navarra',\
                'Ourense','Palencia','Las Palmas','Pontevedra','La Rioja','Salamanca',\
                'Santa Cruz de Tenerife','Segovia','Sevilla','Soria','Tarragona','Teruel','Toledo',\
                'Valencia','València','Valladolid','Zamora','Zaragoza','Ceuta','Melilla']  # Lista de strings a buscar
# Añadir la nueva columna al DataFrame utilizando lambda para pasar search_strings
df['provincia_tags'] = df.apply(lambda row: find_tags_in_text(row, provincia_tags), axis=1)


In [149]:
# search_strings = ['Asturias','Madrid','Málaga']  # Lista de strings a buscar
# df['ccaa_tags'] = df['Contenido'].apply(find_string_in_text)
municipios_tags = list(df_relacion_municipios['municipios'])
# Lista de strings a buscar
# Añadir la nueva columna al DataFrame utilizando lambda para pasar search_strings
df['municipios_tags'] = df.apply(lambda row: find_tags_in_text(row, municipios_tags), axis=1)
#Insert 'df' in column '0'
df.insert(0, "Legislatura", 15, True)
df.insert(0, "Institucion", "Congreso", True)
#'expediente_list'
expediente_list = df.apply(lambda row: row['url'][-len('184/010011'):], axis=1)
df.insert(2, "Expediente", expediente_list, True)
df.insert(8, "Grupo Parlamentario", get_diputado_grupo_parlamentario_listlist(new_new_diputados_dict, df), True)

In [163]:
df["Grupo Parlamentario"] = get_diputado_grupo_parlamentario_listlist(new_new_diputados_dict, df)

In [164]:
df

,Institucion,Legislatura,Expediente,Presentada,Calificada,Contenido,url,diputados_autores,Grupo Parlamentario,comunidades_tags,provincia_tags,municipios_tags
1,Congreso,15,184/000001,03/12/2019,13/12/2019,Discriminación de Asturias en la construcción de líneas ferroviarias en relación con el resto de España.,https://www.congreso.es/es/web/guest/busqueda-de-iniciativas?p_p_id=iniciativas&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_iniciativas_mode=mostrarDetalle&_iniciativas_legislatura=XIV&_iniciativas_id=184/000001,[Isidro ManuelMartínez Oblanca],[],[Asturias],[Asturias],[]
2,Congreso,15,184/000002,03/12/2019,13/12/2019,"Motivos del desinterés del Gobierno por la mejora de los servicios ferroviarios de Asturias, especialmente de Gijón.",https://www.congreso.es/es/web/guest/busqueda-de-iniciativas?p_p_id=iniciativas&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_iniciativas_mode=mostrarDetalle&_iniciativas_legislatura=XIV&_iniciativas_id=184/000002,[Isidro ManuelMartínez Oblanca],[],[Asturias],[Asturias],[Gijón]
3,Congreso,15,184/000003,03/12/2019,13/12/2019,"Previsiones acerca de dotar a los consumidores electrointensivos de un marco jurídico y económico para el suministro eléctrico, así como medidas para frenar la destrucción de empleo en dicho sector.",https://www.congreso.es/es/web/guest/busqueda-de-iniciativas?p_p_id=iniciativas&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_iniciativas_mode=mostrarDetalle&_iniciativas_legislatura=XIV&_iniciativas_id=184/000003,[Isidro ManuelMartínez Oblanca],[],[],[],[]
4,Congreso,15,184/000004,03/12/2019,13/12/2019,Medidas previstas para resolver el problema de seguridad que sufre el paso a nivel que une las parroquias de Veriña y Poago (Gijón).,https://www.congreso.es/es/web/guest/busqueda-de-iniciativas?p_p_id=iniciativas&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_iniciativas_mode=mostrarDetalle&_iniciativas_legislatura=XIV&_iniciativas_id=184/000004,[Isidro ManuelMartínez Oblanca],[],[],[],[Gijón]
5,Congreso,15,184/000005,03/12/2019,13/12/2019,Medidas ante el anuncio de la realización de un ERE de extinción por la empresa Expal Metallurgy en Trubia (Oviedo).,https://www.congreso.es/es/web/guest/busqueda-de-iniciativas?p_p_id=iniciativas&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_iniciativas_mode=mostrarDetalle&_iniciativas_legislatura=XIV&_iniciativas_id=184/000005,[Isidro ManuelMartínez Oblanca],[],[],[],[Oviedo]
...,...,...,...,...,...,...,...,...,...,...,...,...
107967,Congreso,15,184/108065,26/05/2023,01/06/2023,"Dotación presupuestaria incluida en los Presupuestos Generales del Estado para el año 2023 para la ejecución del proyecto de obras de defensa en las Ramblas de Molina en el término municipal de Molina de Segura, Región de Murcia, así como plazos para su ejecución y motivos por los que en los últimos cinco años no se han iniciado.",https://www.congreso.es/es/web/guest/busqueda-de-iniciativas?p_p_id=iniciativas&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_iniciativas_mode=mostrarDetalle&_iniciativas_legislatura=XIV&_iniciativas_id=184/108065,"[Isabel María Borrego Cortés, Juan Luis Pedreño Molina, Antonia López Moya]",[G.P. Popular en el Congreso],[Murcia],[Murcia],"[Segura, Molina de Segura, Murcia]"
107968,Congreso,15,184/108066,27/05/2023,01/06/2023,"Medidas de seguridad en la gestión de los procesos electorales, especialmente en el voto por correo y para evitar posibles fraudes electorales en el futuro.",https://www.congreso.es/es/web/guest/busqueda-de-iniciativas?p_p_id=iniciativas&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_iniciativas_mode=mostrarDetalle&_iniciativas_legislatura=XIV&_iniciativas_id=184/108066,[Pablo Cambronero Piqueras],[G.P. Mixto],[],[],[]
107969,Congreso,15,184/108067,31/05/2023,26/07/2023,Uso de perdigones de plomo en los humedales.,https://www.congreso.es/es/web/guest/busqueda-de-iniciativas?p_p_id=iniciativas&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_iniciativas_mode=mostrarDetalle&_iniciativas_legislatura=XIV&_ini

In [165]:
df['Grupo Parlamentario']

{'G.P. Ciudadanos',
 'G.P. Confederal de Unidas Podemos-En Comú Podem-Galicia en Común',
 'G.P. EH Bildu',
 'G.P. Mixto',
 'G.P. Plural',
 'G.P. Popular en el Congreso',
 'G.P. Republicano',
 'G.P. Socialista',
 'G.P. VOX',
 'G.P. Vasco (EAJ-PNV)'}

In [169]:
df.to_csv(f'congreso_preguntas/out_001133.csv',sep=";", index=False)  